# Learning Resources

Learning resources (courses, videos, articles, etc) can be registered with lsgraph. Once a record has been created, appropriate resources for a skill or user can be recommended or discovered through search.

This notebook will demonstrate:

* creating formats, providers, platforms, resources and offerings 
* organizing access with groups and collections
* finding resources through search and recommendations

If you have not already gone through the setup notebook please do so before running the code below.

In [41]:
from datetime import datetime, timedelta, timezone
import json
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1/"

In [42]:
# Load the credentials set in the setup notebook

with open("credentials.json") as fp:
    headers = json.load(fp)

In [43]:
# Get organization details

r = requests.get(base_url + "organizations/", headers=headers)
print("Status code", r.status_code)

print("Organisation response", r.json())

Status code 200
Organisation response {'organizations': [{'id': '6ff857ca-09ba-4ab5-a252-c65ecdc2d2c7', 'name': 'Test organization'}]}


In [44]:
# Get organization ID for the test organisation created during setup
assert 'Test organization' in [i['name'] for i in r.json()["organizations"]], "Organisation not found, run setup notebook"
organization_id = [i for i in r.json()["organizations"] if i["name"] == "Test organization"][0]["id"]

## Creating new resources

Each resource has a platform and provider. Each resource can have many different offerings (e.g. Spring and Fall semesters). Each offering has a format (e.g. online and classroom based).

In [45]:
# Creating a platform

# Additional information can be stored - see documentation or results below
data = {"name":"ABC", "description":"The ABC learning platform"}

r = requests.post(base_url + f"organizations/{organization_id}/platforms/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
platform_id = results["id"]
print(results)

Status code 200
{'description': 'The ABC learning platform', 'free_trial': None, 'id': '1866fdfc-7262-4f36-943d-73a5b40495be', 'logo': None, 'name': 'ABC', 'subscription': None, 'url': None}


In [46]:
# Creating a provider

# Additional information can be stored - see documentation or results below
data = {"name":"XYZ University", "description":"Educational courses from XYZ University"}

r = requests.post(base_url + f"organizations/{organization_id}/providers/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
provider_id = results["id"]
print(results)

Status code 200
{'description': 'Educational courses from XYZ University', 'id': '9b46cd20-3537-44b0-a7ad-3f30961ac27f', 'logo': None, 'name': 'XYZ University', 'url': None}


In [47]:
# Creating a learning resource

# Additional information can be stored - see documentation or results below
data = {"name":"Learn Python", 
        "short_description":"Get started with this popular programming language",
        "description":"This 5 week course will teach you the basics of Python . . .",
        "provider":provider_id,
        "platform":platform_id,
       }

r = requests.post(base_url + f"organizations/{organization_id}/resources/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
resource_id = results["id"]
print(results)

Status code 200
{'alt_id': None, 'description': 'This 5 week course will teach you the basics of Python . . .', 'id': '27422f23-b451-4834-bcc4-171399973006', 'learning_outcomes': None, 'name': 'Learn Python', 'offerings': [], 'platform': {'description': 'The ABC learning platform', 'free_trial': None, 'id': '1866fdfc-7262-4f36-943d-73a5b40495be', 'logo': None, 'name': 'ABC', 'subscription': None, 'url': None}, 'platform_level': None, 'prerequisite_knowledge': None, 'provider': {'description': 'Educational courses from XYZ University', 'id': '9b46cd20-3537-44b0-a7ad-3f30961ac27f', 'logo': None, 'name': 'XYZ University', 'url': None}, 'retired': False, 'short_description': 'Get started with this popular programming language', 'syllabus': None, 'url': None}


In [48]:
# Create a format

# Additional information can be stored - see documentation or results below
data = {"name":"Online", "description":"Video and online examples available on-demand"}

r = requests.post(base_url + f"organizations/{organization_id}/formats/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
format_id = results["id"]
print(results)

Status code 200
{'description': 'Video and online examples available on-demand', 'id': '00af32cf-4fac-4b94-8db3-29c2bf85f94b', 'logo': None, 'name': 'Online'}


In [49]:
# Create an offering for the new course

start_date = datetime(year=2022, month=1, day=11, hour=9, tzinfo=timezone.utc)
data = {
        "name": "",
        "format": format_id,
        "start_date": start_date.isoformat(),
        "end_date": (start_date + timedelta(days=5 * 7)).isoformat(),
        "pace_min_hrs_per_week": 1,
        "pace_max_hrs_per_week": 2,
        "pace_num_weeks": 5,
        "elapsed_duration": 3024000,
        "min_taught_duration": 5,
        "max_taught_duration": 10,
        "language": "en",
        "cc_language": "en",
        "free": False,
        "free_audit": True,
        "paid": True,
        "certificate": True,
        "quality": 4.7,
        "instructors": "Dr. Smith",
        "retired": False,
    }
r = requests.post(base_url + f"organizations/{organization_id}/resources/{resource_id}/offerings/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
print(results)

Status code 200
{'cc_language': 'en', 'certificate': True, 'elapsed_duration': 3024000.0, 'end_date': '2022-02-15T09:00:00', 'format': {'description': 'Video and online examples available on-demand', 'id': '00af32cf-4fac-4b94-8db3-29c2bf85f94b', 'logo': None, 'name': 'Online'}, 'free': False, 'free_audit': True, 'id': '767fc1e7-5f58-453b-a131-00fc80c0ecd8', 'instructors': 'Dr. Smith', 'language': 'en', 'max_taught_duration': 10.0, 'min_taught_duration': 5.0, 'name': '', 'pace_max_hrs_per_week': 2.0, 'pace_min_hrs_per_week': 1.0, 'pace_num_weeks': 5.0, 'paid': True, 'quality': 4.7, 'retired': False, 'start_date': '2022-01-11T09:00:00'}


## Viewing organization resources

Resources created by an organization can be can be viewed without personalization for individual users.

In [50]:
r = requests.get(base_url + f"organizations/{organization_id}/resources/", headers=headers)
print("Status code", r.status_code)

print("Resources", r.json())

Status code 200
Resources {'resources': [{'alt_id': None, 'description': 'This 5 week course will teach you the basics of Python . . .', 'id': '27422f23-b451-4834-bcc4-171399973006', 'learning_outcomes': None, 'name': 'Learn Python', 'offerings': [{'cc_language': 'en', 'certificate': True, 'elapsed_duration': 3024000.0, 'end_date': '2022-02-15T09:00:00', 'format': {'description': 'Video and online examples available on-demand', 'id': '00af32cf-4fac-4b94-8db3-29c2bf85f94b', 'logo': None, 'name': 'Online'}, 'free': False, 'free_audit': True, 'id': '767fc1e7-5f58-453b-a131-00fc80c0ecd8', 'instructors': 'Dr. Smith', 'language': 'en', 'max_taught_duration': 10.0, 'min_taught_duration': 5.0, 'name': '', 'pace_max_hrs_per_week': 2.0, 'pace_min_hrs_per_week': 1.0, 'pace_num_weeks': 5.0, 'paid': True, 'quality': 4.7, 'retired': False, 'start_date': '2022-01-11T09:00:00'}], 'platform': {'description': 'The ABC learning platform', 'free_trial': None, 'id': '1866fdfc-7262-4f36-943d-73a5b40495be', 

The result includes details on platform, provider, offerings and format

## Organizing resource access

The resources available to a user when searching or requesting recommendations can be controlled using groups and collections.

Users can be added to groups. Resources can be added to collections. Groups and users can be added to collections enabling access to the collection resources.

In [51]:
# Fetching a user to use in testing

r = requests.get(base_url + f"organizations/{organization_id}/users/", headers=headers)
print("Status code", r.status_code)

print("Users", r.json())

user_id = r.json()["users"][0]["id"]

Status code 200
Users {'users': [{'email': 'Adam@test.local', 'id': 'b18385f1-7bed-4b98-a8e0-d3b3e7f3f849', 'name': 'Adam', 'profile': 'b856a23e-bdc2-436c-96ec-4d6d9dc486d3'}, {'email': 'Beth@test.local', 'id': '2af63b72-9a9c-4266-9ca5-00fcbfb007c2', 'name': 'Beth', 'profile': '65e035c9-02ef-4ef8-9c66-29808c1dd565'}, {'email': 'Chris@test.local', 'id': '3945852c-f3bd-45c0-b608-72c449d28387', 'name': 'Chris', 'profile': 'a0ec621d-1acb-4445-8451-29eb7e352ec5'}, {'email': 'Dani@test.local', 'id': '80e6b375-209b-48df-b4b1-55bf16558ea9', 'name': 'Dani', 'profile': 'b35dbf53-8d07-4c6d-8e0e-3deb30556f95'}]}


In [52]:
# Create a group

data = {"name":"Test group",}

r = requests.post(base_url + f"organizations/{organization_id}/groups/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
group_id = results["id"]
print(results)

Status code 200
{'id': 'd1d0990b-6a87-4c7b-8fd7-6f8a9509a7c5', 'members': [], 'name': 'Test group', 'whole_organization': False}


In [53]:
# Add our user to the group

data = {"members":[{"id":user_id},]}

r = requests.post(base_url + f"organizations/{organization_id}/groups/{group_id}/members/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
print(results)

Status code 200
{'members': [{'id': 'b18385f1-7bed-4b98-a8e0-d3b3e7f3f849', 'name': 'Adam'}]}


In [54]:
# Create a collection

data = {"name":"Test collection",}

r = requests.post(base_url + f"organizations/{organization_id}/collections/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
collection_id = results["id"]
print(results)

Status code 200
{'id': '61297db9-1bf1-4fa2-8821-207e8b632ad6', 'name': 'Test collection', 'public': False}


In [55]:
# Add our group to the collection

data = {"members":[{"group_id":group_id},]}

r = requests.post(base_url + f"organizations/{organization_id}/collections/{collection_id}/members/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()
print(results)

Status code 200
{'members': [{'edit': False, 'group_id': 'd1d0990b-6a87-4c7b-8fd7-6f8a9509a7c5'}]}


In [56]:
# Create resources and add to collection

for title in ["Learn Requests", "Learn Flask", "Learn Jupyter"]:
    # Creating a minimal resource
    data = {"name":title, 
            "short_description":"",
            "description":"",
            "provider":provider_id,
            "platform":platform_id,
           }

    r = requests.post(base_url + f"organizations/{organization_id}/resources/", 
                      json=data, headers=headers)
    print("Status code", r.status_code)
    
    # Add resource to collection
    if title == "Learn Requests":
        # Skip one resource to test collection filtering
        continue
    data = {"resources":[{"resource_id":r.json()["id"]},]}

    r = requests.post(base_url + f"organizations/{organization_id}/collections/{collection_id}/resources/", 
                      json=data, headers=headers)
    print("Status code", r.status_code)

    results = r.json()
    print(results)

Status code 200
Status code 200
Status code 200
{'resources': [{'resource_id': '1714ff11-07b2-47c5-8cc7-1f498a6bc6b3'}]}
Status code 200
Status code 200
{'resources': [{'resource_id': '140a11ab-fedc-41e0-8ad2-0acecbefa17a'}]}


## Search and recommendations for users

In [61]:
r = requests.get(base_url + f"organizations/{organization_id}/resources/?user={user_id}", headers=headers)
print("Status code", r.status_code)

results = r.json()["resources"]

for i in results:
    print(i["name"])

Status code 200
Learn Jupyter
Learn Flask


Only the "Learn Jupyter" and "Learn Flask" resources are returned.

As the "Learn Requests" resource is not connected to the user it is not returned.

In [62]:
# Adding search

r = requests.get(base_url + f"organizations/{organization_id}/resources/?user={user_id}&query=Jupyter", headers=headers)
print("Status code", r.status_code)

results = r.json()["resources"]

for i in results:
    print(i["name"])

Status code 200
Learn Jupyter
